# Módulo de Programação Python

# Trilha Python - Aula 17 e 18: Utilizando Pandas - Avançado

<img align="center" style="padding-right:10px;" src="Figuras/aula-18_fig_01.png">

__Objetivo__:  Trabalhar com pacotes e módulos disponíveis em python: Pandas:  Recursos __Pandas__ de alto desempenho. Persistência de dados com __Pandas__.

# Recursos Pandas de alto desempenho: eval() e query()

Como já vimos nas seções anteriores, o poder da __Python__ na análise e processamento de dados  é construído sobre a capacidade do __NumPy__ e do __Pandas__ de enviar operações básicas para C por meio de uma sintaxe intuitiva. A modo de exemplos temos as operações vetorizadas/transmitidas em __NumPy__ e operações do tipo agrupamento em __Pandas__.

Embora essas abstrações sejam eficientes e eficazes para muitos casos de uso comuns, elas geralmente dependem da criação de objetos intermediários temporários, o que pode causar sobrecarga indevida no tempo computacional e no uso da memória.

A partir da versão 0.13 (lançada em janeiro de 2014), o __Pandas__ inclui algumas ferramentas experimentais que permitem acessar diretamente operações C-speed sem alocação dispendiosa de arrays intermediários.
Estas são as funções ``eval()`` e ``query()``.

## Motivando ``query()`` e ``eval()``: Expressões Compostas

Vimos anteriormente que __NumPy__ e __Pandas__ suportam operações vetorizadas rápidas como, por exemplo, ao adicionar os elementos de duas matrizes.

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
402 µs ± 9.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Conforme discutido em anteriormente, este método é muito mais rápido do que fazer a adição por meio de um loop __Python__.

Mas esta abstração pode se tornar menos eficiente ao calcular expressões compostas. Por exemplo, considere a seguinte expressão.

In [3]:
mask = (x > 0.5) & (y < 0.5)
mask.shape

(1000000,)

Como o __NumPy__ avalia cada subexpressão, isso é aproximadamente equivalente a:

In [ ]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

Em outras palavras, cada etapa intermediária é alocada explicitamente na memória. Se os arrays ``x`` e ``y`` forem muito grandes, isso pode levar a memória significativa e sobrecarga computacional.

Por outro lado, a biblioteca __Numexpr__ oferece a capacidade de calcular esse tipo de expressão composta elemento por elemento, sem a necessidade de alocar matrizes intermediárias completas.

A [documentação do Numexpr](https://github.com/pydata/numexpr) tem mais detalhes, mas por enquanto é suficiente dizer que a biblioteca aceita uma *string* fornecendo a expressão no estilo NumPy que você deseja calcular.

In [4]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

In [7]:
%timeit (x > 0.5) & (y < 0.5)

343 µs ± 3.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%timeit numexpr.evaluate('(x > 0.5) & (y < 0.5)')

890 µs ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


A função ``allclose`` retorna ``True`` se duas matrizes forem iguais elemento por elementos dentro de uma margem de tolerância.
```
    numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
```

A vantagem aqui é que o __Numexpr__ avalia a expressão de uma forma que não usa arrays temporários de tamanho normal e, portanto, pode ser muito mais eficiente que o __NumPy__, especialmente para arrays grandes.

As ferramentas ``eval()`` e ``query()`` do Pandas que discutiremos aqui são conceitualmente semelhantes e dependem do pacote __Numexpr__.

## ``pandas.eval()`` para operações eficientes

A função ``eval()`` no __Pandas__ usa expressões de _string_ para calcular operações de forma eficiente usando ``DataFrame``s.
Por exemplo, considere os seguintes ``DataFrame``s:

In [10]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

Para calcular a soma de todos os quatro ``DataFrame``s usando a abordagem típica do __Pandas__, podemos simplesmente escrever a soma.

In [11]:
%timeit df1 + df2 + df3 + df4

19.5 ms ± 485 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


O mesmo resultado pode ser calculado via ``pd.eval`` construindo a expressão como uma string.

In [12]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10.1 ms ± 97.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


A versão ``eval()`` desta expressão é cerca de 50% mais rápida (e usa muito menos memória), dando o mesmo resultado.

In [13]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

### Operações suportadas por ``pd.eval()``

A partir do Pandas v0.16, ``pd.eval()`` suporta uma ampla gama de operações.
Para demonstrá-los, usaremos os seguintes inteiros ``DataFrame``s

In [14]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### Operadores aritméticos
``pd.eval()`` suporta todos os operadores aritméticos.

In [15]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Operadores de comparação

``pd.eval()`` suporta todos os operadores de comparação, incluindo expressões encadeadas.

In [16]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Operadores bit a bit

``pd.eval()`` suporta os operadores bit a bit ``&`` e ``|``

In [17]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

Além disso, ele suporta o uso dos literais ``and`` e ``or`` em expressões booleanas.

In [18]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Atributos e índices de objetos

``pd.eval()`` suporta acesso a atributos de objetos através da sintaxe ``obj.attr`` e índices através da sintaxe ``obj[index]``

In [19]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

## ``DataFrame.eval()`` para operações em colunas

Assim como o __Pandas__ tem uma função ``pd.eval()`` de nível superior, os ``DataFrame``s têm um método ``eval()`` que funciona de maneira semelhante.

A vantagem do método ``eval()`` é que as colunas podem ser referenciadas pelo nome.

In [20]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Usando ``pd.eval()`` como acima, podemos calcular expressões com as três colunas.

In [21]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

O método ``DataFrame.eval()`` permite uma avaliação muito mais sucinta de expressões com as colunas.

In [22]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Repare aqui que tratamos nomes de colunas como variáveis dentro da expressão avaliada, e o resultado é o que estávamos esperando.

### Atribuição em DataFrame.eval()

Além das opções que acabamos de discutir, ``DataFrame.eval()`` também permite atribuição a qualquer coluna.

Vamos usar o ``DataFrame`` de antes, que tem as colunas ``'A'``, ``'B'`` e ``'C'``

In [23]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Podemos usar ``df.eval()`` para criar uma nova coluna ``'D'`` e atribuir a ela um valor calculado a partir das outras colunas.

In [24]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


Da mesma forma, qualquer coluna existente pode ser modificada.

In [25]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### Variáveis locais em DataFrame.eval()

O método ``DataFrame.eval()`` suporta uma sintaxe adicional que permite trabalhar com variáveis locais do __Python__.

In [28]:
df.mean(1)

0      0.100740
1     -0.154913
2      0.534579
3     -0.036899
4      0.500769
         ...   
995    0.165847
996    0.100385
997    0.698249
998    0.571263
999    0.008088
Length: 1000, dtype: float64

In [26]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

O caractere ``@`` aqui marca um nome de variável em vez de um nome de coluna, e permite avaliar eficientemente expressões envolvendo os dois "namespaces": o namespace de colunas e o namespace de objetos __Python__.

Observe que este caractere ``@`` só é suportado pelo método ``DataFrame.eval()``, e não pela função ``pandas.eval()``. Isto porque o ``pandas.eval ()`` função só tem acesso ao namespace __Python__.

## Método DataFrame.query()

O ``DataFrame`` possui outro método baseado em avaliação de _strings_, chamado método ``query()``.

In [29]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

Assim como no exemplo usado em nossa discussão sobre ``DataFrame.eval()``, esta é uma expressão envolvendo colunas do ``DataFrame``. Entretanto, ele não pode ser expresso usando a sintaxe ``DataFrame.eval()``!

Em vez disso, para este tipo de operação de filtragem, você pode usar o método ``query()``

In [30]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

Além de ser um cálculo mais eficiente, comparado à expressão de mascaramento é muito mais fácil de ler e entender.
Observe que o método ``query()`` também aceita a flag ``@`` para marcar variáveis locais.

In [31]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Desempenho: quando usar essas funções

Ao ponderar se essas funções devem ser usadas, há duas considerações: _tempo de computação_ e _uso de memória_.

O uso da memória é o aspecto mais previsível. Como já mencionado, toda expressão composta envolvendo arrays __NumPy__ ou ``DataFrame``s do __Pandas__ resultará na criação implícita de arrays temporários.

In [32]:
# Isto ...
x = df[(df.A < 0.5) & (df.B < 0.5)]

In [33]:
# ... é aproximadamente equivalente a isso
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

Se o tamanho dos ``DataFrame``s temporários for significativo, comparado à memória disponível do sistema, então é uma boa ideia usar uma expressão ``eval()`` ou ``query()``.

Você pode verificar o tamanho aproximado do seu array em bytes.

In [34]:
df.values.nbytes

32000

Do lado do desempenho, ``eval()`` pode ser mais rápido mesmo quando você não está maximizando a memória do sistema.

A questão é como seus ``DataFrame``s temporários são em relação ao tamanho do cache L1 ou L2 da CPU em seu sistema. Se eles forem muito maiores, então ``eval()`` pode evitar algum movimento potencialmente lento de valores entre os diferentes caches de memória.

Na prática, acho que a diferença no tempo de computação entre os métodos tradicionais e o método ``eval``/``query`` geralmente não é significativa – na verdade, o método tradicional é mais rápido para arrays menores!

O benefício de ``eval``/``query`` está principalmente na memória salva e na sintaxe às vezes mais limpa que eles oferecem.

Cobrimos a maioria dos detalhes de ``eval()`` e ``query()`` aqui. Para obter mais informações, você pode consultar a documentação do __Pandas__, em particular veja a [seção "Aprimorando o desempenho"](http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html).